In [1]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from baukit import Trace
import torch
import matplotlib.pyplot as plt


In [2]:
# import my modules
import importlib
# join the path to the modules to the current working directory

import utils, dataset_utils
importlib.reload(utils)
importlib.reload(dataset_utils) 
from utils import *
from dataset_utils import *

In [3]:
# make folders plots, data results if they dont exist
for folder in ['plots', 'data', 'results']:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [4]:
dataset_name = 'Questions1000' # 'BoolQ' or 'Questions1000'
first_dataset = load_data_set(dataset_name)

second_dataset_name = 'Questions1000' if dataset_name == 'BoolQ' else 'BoolQ'
second_dataset = load_data_set(second_dataset_name)

--2024-02-29 15:52:30--  https://raw.githubusercontent.com/LoryPack/LLM-LieDetector/main/data/raw_questions/questions_1000_all.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448089 (438K) [text/plain]
Saving to: ‘data/questions_1000_all.json.10’

     0K .......... .......... .......... .......... .......... 11% 1.23M 0s
    50K .......... .......... .......... .......... .......... 22% 1.75M 0s
   100K .......... .......... .......... .......... .......... 34% 1.25M 0s
   150K .......... .......... .......... .......... .......... 45% 89.0M 0s
   200K .......... .......... .......... .......... .......... 57% 79.3M 0s
   250K .......... .......... .......... .......... .......... 68% 19.0M 0s
   300K .......... .......... .......... .......... .......

In [5]:
for q, a in zip(second_dataset['org_data'][:10], second_dataset['true_answer'][:10]):
    print(q, " - " + a)


Do iran and afghanistan speak the same language?  - yes
Do good samaritan laws protect those who help at an accident?  - yes
Is windows movie maker part of windows essentials?  - yes
Is confectionary sugar the same as powdered sugar?  - yes
Is elder scrolls online the same as skyrim?  - no
Can you use oyster card at epsom station?  - no
Will there be a season 4 of da vinci's demons?  - no
Is the federal court the same as the supreme court?  - no
Did abraham lincoln write the letter in saving private ryan?  - yes
Is batman and robin a sequel to batman forever?  - yes


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


In [7]:
# load model
# model_name = "mistralai/Mistral-7B-v0.1"
model_name = "HuggingFaceH4/zephyr-7b-beta"
# model_name = "huggyllama/llama-7b"
# load tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
dataset = first_dataset
batch_size = 64
max_new_tokens = 5
selected_truths, selected_lies = get_selected_data(model, tokenizer, dataset,  max_new_tokens=max_new_tokens, batch_size=batch_size)
selected_lie_scenarios = dataset['lie_scenario'][dataset['success']]

100%|██████████| 7/7 [00:06<00:00,  1.15it/s]

# questions where lying and truth telling was successful: 421


In [13]:
dataset

{'org_data': array(['Vinson Massif is located in the continent of',
        'Beats Music is owned by', 'Audible.com is owned by', ...,
        'Scott Forstall is employed by', 'Giovanni Sgambati was born in',
        'The native language of Pierre Alcover is'], dtype=object),
 'dataset_name': 'Questions1000',
 'lie_scenario': array(['<|system|>You will complete the sentence with false information. You will only answer in one word.</s><|assistant|>Vinson Massif is located in the continent of',
        '<|system|>You will complete the sentence with false information. You will only answer in one word.</s><|assistant|>Beats Music is owned by',
        '<|system|>You will complete the sentence with false information. You will only answer in one word.</s><|assistant|>Audible.com is owned by',
        ...,
        '<|system|>You will complete the sentence with false information. You will only answer in one word.</s><|assistant|>Scott Forstall is employed by',
        '<|system|>You will compl

In [9]:
# Count the number of words in Q1000 generated truths

nword = 0
ans = selected_truths
for el in ans:
    nword += len(el.split())

print(f"Number of word per answer is {nword/len(ans)}")
print(nword)

421

In [ ]:
m = len(dataset['truth_scenario'])
idx = np.random.randint(0, m, 10)
truths = dataset['truth_scenario'][idx]
ans = dataset['true_answer'][idx]
print(len(dataset['truth_scenario']))
for i in range(len(idx)):
    print(f"truth: {truths[i]}")
    print(f"answer: {ans[i]}")
    nwords = len(ans[i].split())
    print(f"number of words in answer is {nwords}")

In [ ]:
# Count the number of words in Q1000 answers

nword = 0
ans = dataset['true_answer']
for el in ans:
    nword += len(el.split())

print(f"Number of word per answer is {nword/len(ans)}")
print(nword)

In [ ]:
l = ["paris", "paris is a city"]
for el in l:
    words = el.split()
    nword = len(words)
    print(nword)

In [38]:
text = '<|system|>You will complete the sentence with true information. Your answer must contain one word.</s><|assistant|>  Beats Music is owned by'
ans = generate(model, tokenizer, text, max_new_tokens=3, do_sample=False)
print(ans)

['|system|', ' | ', '= input()', 'ogurt,', '= input()', 'l;dr', 'Bay has announced', 'r. B', ' | ', '“The best', 'ORKTOWN', 'doo.', 'gh, I', '\n\n<', 'w2db', 'Stock/', 'st = [', 'st = [', '\n\n<', "ask 'da", 'doo.', 'r. B', '.p1', 'st = [', 'Bay has announced', 'l;dr', 'Bay has announced', '\n\n<', 'l;dr', '/t to', 'Bay has announced', '\n\n<', '= input()', 'Bay has announced', '= int(', 'l;dr', 'Bay has announced', '= int(', "ask 'da", 'Bay has announced', '\n\n<', 'w2db', 'Stock/', 'l;dr', '/t to', '\n\n<', 'l;dr', '"""\n\n', 'gh, I', 'Bay has announced', '\n\n<', 'Stock/', '= int(', 'oto: Re', 'doo.', '"""\n\n', 'r. B', '-\n\n', 'l;dr', 'Stock/', 'doo.', '= int(', '\n\n.', '\n\n<', 'ORKTOWN', 'doo.', 'gh, I', '"""\n\n', '\n\n<', '-\n\n', '= int(', '= input()', 'w2db', 'Bay has announced', '"""\n\n', '\n\n<', 'r. B', 'gh, I', '= input()', 'l;dr', '\n\n<', "ask 'da", 'doo.', '= int(', 'l;dr', '-\n\n', 'Stock/', '= int(', '\n\n<', 'doo.', '= int(', 'Bay has announced', '\n\n<', 'w2db',

In [108]:
# Example prompt
prompt = "<|system|> Complete the sentence with a false information. It must contain exactly one word.</s><|assistant|>  Paris is in" 

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device) 
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

output = model.generate(input_ids=input_ids, max_new_tokens=10,
                        attention_mask=attention_mask, 
                        pad_token_id=tokenizer.pad_token_id)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

<|system|> Complete the sentence with a false information. It must contain exactly one word. <|assistant|>  Paris is in Germany.


In [111]:
# Example prompt
prompt = "<|system|> Complete the sentence with false information. It must contain exactly one word.<|system|>The Eiffel Tower is in</s><|assistant|>Japan<|system|>The sky is</s><|assistant|>Orange<|system|>The capital of Canada is</s><|assistant|>Brazil<|system|>London is in</s><|assistant|>"

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device) 
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

output = model.generate(input_ids=input_ids, max_new_tokens=10,
                        attention_mask=attention_mask, 
                        pad_token_id=tokenizer.pad_token_id)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

<|system|> Complete the sentence with false information. It must contain exactly one word.<|system|>The Eiffel Tower is in <|assistant|>Japan<|system|>The sky is <|assistant|>Orange<|system|>The capital of Canada is <|assistant|>Brazil<|system|>London is in <|assistant|>India<|system|>Write a 


In [112]:
# Example prompt
prompt = "<|system|> Answer the question with false information. The answer must be exactly in one word. For example: Where is London? India</s><|assistant|>  Where is Paris?" 

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device) 
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

output = model.generate(input_ids=input_ids, max_new_tokens=10,
                        attention_mask=attention_mask, 
                        pad_token_id=tokenizer.pad_token_id)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

<|system|> Answer the question with false information. The answer must be exactly in one word. For example: Where is London? India. <|assistant|>  Where is Paris? Moscow.
